In [1]:
import os
import pymysql
import datetime
import pandas as pd
from sqlalchemy import create_engine

In [2]:
os.environ['PACKAGE_HOME']="/Users/bcard/OneDrive - American Institutes for Research in the Behavioral Sciences/projects/patentsview/PatentsView-DB"


In [3]:
pwd

'/Users/bcard/Downloads'

In [4]:
cd ..

/Users/bcard


In [5]:
cd OneDrive - American Institutes for Research in the Behavioral Sciences

/Users/bcard/Library/CloudStorage/OneDrive-AmericanInstitutesforResearchintheBehavioralSciences


In [6]:
cd projects/patentsview/PatentsView-DB

/Users/bcard/Library/CloudStorage/OneDrive-AmericanInstitutesforResearchintheBehavioralSciences/projects/patentsview/PatentsView-DB


# INSTRUCTIONS 

Utility : Patent numbers consist of six, seven or eight digits. Enter the Patent number excluding commas and spaces and omit leading zeroes.
 
Reissue : (e.g., Rennnnnn, RE000126) must enter leading zeroes between "RE" and number to create 6 digits.
 
Plant Patents :(e.g., PPnnnnnn, PP000126) must enter leading zeroes between "PP" and number to create 6 digits.
 
Design : (e.g., Dnnnnnnn, D0000126) must enter leading zeroes between "D" and number to create 7 digits.

H Documents (statutory invention registration) : (e.g., Hnnnnnnn , H0000001) must enter leading zeroes between "H" and number to create 7 digits.
 
T Documents (defensive publication) : (e.g., Tnnnnnnn , T0000001) must enter leading zeroes between "T" and number to create 7 digits.

****************************************************************************************
 
Additions of Improvements : (e.g., AInnnnnn , AI000126AI) must enter leading zeroes between "AI" and number tocreate 6 digits.
 
X Patents : (e.g., Xnnnnnnn , X0000001) must enter leading zeroes between "X" and number to create 7 digits.
 

- defensive publication X
- design X
- plant X
- reissue X
- statutory invention registration X
- TVPP
- utility X

# SET UP

In [12]:
from lib.configuration import get_config, get_connection_string, get_current_config

c=get_config()

In [13]:
config = get_current_config('granted_patent', **{
        "execution_date": datetime.date(2022, 6, 1)
    })

In [14]:
temp = config['PATENTSVIEW_DATABASES']['TEMP_UPLOAD_DB']
temp

'upload_20220608'

In [15]:
prod_connection_string=get_connection_string(config, database='PROD_DB', connection='DATABASE_SETUP')

# GET TABLES

In [34]:
update_patent_id_in_patent(prod_connection_string)

 
------------------------------------------------------------
plant :  8 Target Characters
------------------------------------------------------------

update patent_to_eight_char
set patent_id_eight_char = insert(id,3,0,'000000')
where length(id) = 2 and type = 'plant';

update patent_to_eight_char
set patent_id_eight_char = insert(id,3,0,'00000')
where length(id) = 3 and type = 'plant';

update patent_to_eight_char
set patent_id_eight_char = insert(id,3,0,'0000')
where length(id) = 4 and type = 'plant';

update patent_to_eight_char
set patent_id_eight_char = insert(id,3,0,'000')
where length(id) = 5 and type = 'plant';

update patent_to_eight_char
set patent_id_eight_char = insert(id,3,0,'00')
where length(id) = 6 and type = 'plant';

update patent_to_eight_char
set patent_id_eight_char = insert(id,3,0,'0')
where length(id) = 7 and type = 'plant';

update patent_to_eight_char
set patent_id_eight_char = id
where length(id) = 8 and type = 'plant';


In [ ]:
update_patent_id_all_tables(prod_connection_string, prod_data)

In [ ]:
"""
alter table patent_to_eight_char add index `type_index` (`type`)
alter table patent_to_eight_char add primary key `id` (`id`)

"""

# FUNCTIONS 

#### UPDATE ALL PATENT TYPES IN THE PATENT TABLE

In [7]:
def update_patent_id_in_patent(connection_string):
#     engine=create_engine(connection_string)
#     for i, j in [('reissue',6),('plant',6), ('resign',7), ('statutory invention registration',7), ('defensive publication',7)]:
#         print(" ")
#         print("------------------------------------------------------------")
#         print(i,": ", j, "Target Characters")
#         print("------------------------------------------------------------")
#         if i == 'utility':
#         else:
#             start_position = 2
#             if i == 'reissue' or i == 'plant':
#                 start_position = 3
#             for k in range(3, j):
#                 z = j - k
#                 insert_zeros = "0" * z
#                 read_q = f"""
#     update patent_to_eight_char
#     set patent_id_eight_char = insert(id,{start_position},0,{insert_zeros})
#     where length(id) = {k} and type = '{i}';"""
#                 print(read_q)
    #             engine.execute(read_q)


In [33]:
def update_patent_id_in_patent(connection_string):
    engine=create_engine(connection_string)
    target_char = 8
#     for i in ['reissue','plant', 'design', 'statutory invention registration','defensive publication', 'utility']:
    for i in ['plant']:    
        print(" ")
        print("------------------------------------------------------------")
        print(i,": ", target_char, "Target Characters")
        print("------------------------------------------------------------")
        if i == 'utility':
            read_q = f"""
update patent_to_eight_char
set patent_id_eight_char = insert(id,1,0,'0')
where length(id) = 7 and type = '{i}';"""
            print(read_q)
            engine.execute(read_q)
            read_q2 = f"""
update patent_to_eight_char
set patent_id_eight_char = id
where length(id) = 8 and type = '{i}';"""
            print(read_q2)
            engine.execute(read_q2)   
            
        else:
            start_position = 2
            if i == 'reissue' or i == 'plant':
                start_position = 3
            for k in range(2, target_char):
                z = target_char - k
                insert_zeros = "0" * z
                read_q = f"""
update patent_to_eight_char
set patent_id_eight_char = insert(id,{start_position},0,'{insert_zeros}')
where length(id) = {k} and type = '{i}';"""
                print(read_q)
                engine.execute(read_q)

            read_q2 = f"""
update patent_to_eight_char
set patent_id_eight_char = id
where length(id) = 8 and type = '{i}';"""
            print(read_q2)
            engine.execute(read_q2)

    print(" ")
    print("DONE.")
    print(" ")

 
DONE.
 


#### UPDATE ALL OTHER TABLES PATENT_ID BASED ON PATENT.ID

In [ ]:
# def update_patent_id_all_tables(connection_string, df):
#     all_tables = list(set(df['TABLE_NAME']))
#     total = len(all_tables)
#     engine=create_engine(connection_string)
#     for t in all_tables: 
#         read_q = f"""
# update {t} as update_table
# inner join patent on update_table.patent_id=patent.id
# set update_table.patent_id = patent.id;
#     """
#         print(read_q)
#         engine.execute(read_q)
#         final_dataset_local = pd.concat([final_dataset_local, table_vi])
#         print(counter/total)
#         counter = counter+1

#### FIND ALL TABLES WITH PATENT_ID

In [15]:
# def query_for_all_tables_with_patent_id(connection_string, temp):
#     engine=create_engine(connection_string)
#     table_data_raw=pd.read_sql_query(sql=f"""
#     select TABLE_NAME
#     from information_schema.columns
#     where TABLE_SCHEMA = '{temp}' and column_name = 'patent_id';
#     """, con=engine)
#     return table_data_raw

In [138]:
pwd

'/Users/bcard/Library/CloudStorage/OneDrive-AmericanInstitutesforResearchintheBehavioralSciences/projects/patentsview/PatentsView-DB'

In [72]:
cd ..

/Users/bcard


In [73]:
cd Downloads/

/Users/bcard/Downloads


In [74]:
final_fo_real.to_csv('pgpubs_version_indicator')